In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from cdec_maps import cdec

In [ ]:
c=cdec.Reader()

In [ ]:
df=c.read_daily_stations()
df.head()

In [ ]:
import hvplot.pandas
import holoviews as hv
from holoviews import opts, dim

In [ ]:
map = df.hvplot.points(x='Longitude',y='Latitude',hover_cols='all',geo=True,tiles='OSM')
map

In [ ]:
print(map)

In [ ]:
from holoviews import streams
selection = streams.Selection1D(source=map.points.I, index=[0])

In [ ]:
station_meta_info = c.read_all_stations_meta_info()

In [ ]:
def show_station_info(index):
    if index == None or len(index) == 0:
        index = [0]
    dflist = [c.read_station_meta_info(id)[1] for id in df.iloc[index].ID]
    return hv.Layout([hv.Table(df) in dflist]).cols(1)

In [ ]:
dmap_show_table = hv.DynamicMap(show_station_info, streams=[selection])

In [ ]:
show_station_info([0])

In [ ]:
import geopandas as gpd
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

In [ ]:
gdf

In [ ]:
from shapely.geometry import Polygon, LineString, Point
delta_region=Polygon([(-123,39), (-119,39), (-119,37), (-123,37)])

In [ ]:
gdf[gdf.within(delta_region)]

In [ ]:
delta_boundary=gpd.read_file('./Legal_Delta_Boundary.geojson')

In [ ]:
delta_boundary.crs

In [ ]:
gdf=gdf.set_crs(epsg=4326)

In [ ]:
delta_boundary.geometry[0]

In [ ]:
delta_gdf=gdf[gdf.within(delta_boundary.geometry[0])]

In [ ]:
len(list(delta_gdf.ID))

In [ ]:
rdf=c.read_realtime_stations()

In [ ]:
rgdf = gpd.GeoDataFrame(rdf, geometry=gpd.points_from_xy(rdf.Longitude, rdf.Latitude))

In [ ]:
rdelta_gdf=rgdf[rgdf.within(delta_boundary.geometry[0])]

In [ ]:
print(list(rdelta_gdf[~rdelta_gdf.ID.isin(delta_gdf.ID)].ID))